# Finetuning do Modelo MISTRAL utilizando o dataset "TheAmazonTitles-1.3MM".

Objetivos:


*   Receber perguntas com um contexto obtido por meio do arquivo json
“trn.json” que está contido dentro do dataset.

*   O modelo deverá gerar uma resposta baseada na pergunta do
usuário trazendo como resultado do aprendizado do fine-tuning os
dados da sua descrição.



Este código implementa um modelo de linguagem utilizando transformers e bibliotecas como unsloth e trl para treinamento e adaptação eficiente com LoRA. Inclui a preparação e filtragem de um conjunto de dados, quantização do modelo em 4 bits, e um pipeline de treinamento otimizado para reduzir o uso de memória e melhorar a performance.

O código demonstra um fluxo para treinar e adaptar um modelo de linguagem, utilizando técnicas de quantização e adaptação de parâmetros (LoRA). Com dados filtrados e ajuste de hiperparâmetros, o modelo é treinado para responder a prompts em um formato padrão, aproveitando recursos de hardware de forma otimizada.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
TRAIN_DATASET_URL='/content/drive/MyDrive/FIAP/TC4/trn.json'
DATASET_FOLDER='/content/drive/MyDrive/FIAP/TC4/'
TEST_DATASET_URL='/content/drive/MyDrive/FIAP/TC4/tst.json'

Instalação das bibliotecas unsloth, xformers, trl, peft, accelerate, bitsandbytes, datasets, triton, nltk, evaluate e transformers, que servem para processamento de linguagem natural, otimização de modelos, manipulação de dados e aceleração de aprendizado de máquina.






In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install datasets
!pip install triton
!pip install nltk
!pip install evaluate
!pip install transformers

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-d8lunrj0/unsloth_3958bcc3aa95448faff0806a139c7a39
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-d8lunrj0/unsloth_3958bcc3aa95448faff0806a139c7a39
  Resolved https://github.com/unslothai/unsloth.git to commit c3f4e9a87d964ecee1efd9963f497119edbefaab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
 

Importação das bibliotecas FastLanguageModel, is_bfloat16_supported, torch, json, datasets, trl, transformers e pandas, utilizadas para manipulação de modelos, treinamento e carregamento de datasets, com variáveis definidas para controle de sequência e precisão do carregamento.








In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import pandas as pd

max_seq_length = 2048
dtype = None
load_in_4bit = True

Carregamento do conjunto de dados de treino com load_dataset, utilizando apenas 5% do dataset (train[:5%]), equivalente a 100 mil linhas, para evitar exceder a capacidade e otimizar o uso da GPU disponível.

Usamos apenas 5% do dataset, com 100 mil linhas.

In [ ]:
train_dataset = load_dataset(DATASET_FOLDER, split='train[:5%]', data_files="trn.json")

Aplicação de um filtro no train_dataset para remover exemplos onde o campo 'content' está vazio, garantindo que apenas dados válidos e relevantes sejam utilizados durante o treinamento do modelo.








In [ ]:
train_dataset = train_dataset.filter(lambda example: example['content'] != '')

Importação de módulos do transformers para tokenização e modelos de linguagem, além de BitsAndBytesConfig para configurar a quantização em 4 bits, reduzindo o consumo de memória e otimizando o uso da GPU.








In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

Importação de FastLanguageModel e is_bfloat16_supported do unsloth, além de SFTConfig, SFTTrainer e DataCollatorForCompletionOnlyLM do trl, que são utilizados para configurar, treinar e manipular modelos de linguagem de forma eficiente.








In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

Carregamento do modelo de linguagem causal (AutoModelForCausalLM) e do tokenizador (AutoTokenizer) pré-treinados, que permite a utilização de um modelo otimizado em 4 bits para geração de texto.








In [ ]:
model = AutoModelForCausalLM.from_pretrained("unsloth/mistral-7b-v0.3-bnb-4bit")
tokenizer = AutoTokenizer.from_pretrained("unsloth/mistral-7b-v0.3-bnb-4bit")

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Definição da função formatting_prompts_func para formatar prompts a partir dos exemplos fornecidos, criando uma estrutura de texto padrão para perguntas e respostas. A função gera textos no formato "### Question: {título}\n ### Answer: {conteúdo}" e os armazena em uma lista. Além disso, é definida a variável response_template, que fornece um formato base para as respostas ("### Answer:").








In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example)):
        text = f"### Question: {example['title'][i]}\n ### Answer: {example['content'][i]}"
        output_texts.append(text)
    return output_texts

response_template = " ### Answer:"

In [ ]:
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post2: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Configuração do modelo FastLanguageModel com get_peft_model para otimização com LoRA, adaptando parâmetros para uso eficiente de memória. Em seguida, o SFTTrainer é usado para treinar o modelo com o dataset especificado, utilizando configurações detalhadas de treinamento (TrainingArguments), incluindo tamanho de batch, acumulação de gradiente, taxa de aprendizado e tipo de otimização, visando um treinamento eficiente e otimizado em termos de memória.








In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field="content",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 25,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: Already have LoRA adapters! We shall skip this step.
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
training = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 84,523 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 25
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.852100
2,0.652100
3,0.973400
4,0.941900
5,0.584000
6,0.880500
7,0.831700
8,0.917900
9,0.820000
10,0.623800


In [ ]:
training

TrainOutput(global_step=25, training_loss=0.895662739276886, metrics={'train_runtime': 107.7948, 'train_samples_per_second': 1.855, 'train_steps_per_second': 0.232, 'total_flos': 3101501918625792.0, 'train_loss': 0.895662739276886, 'epoch': 0.0023661918508352657})

In [ ]:
format_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


Tokenização do prompt usando o tokenizer para gerar tensores em PyTorch (return_tensors="pt"), colocando-os na GPU (to("cuda")). O TextStreamer do transformers é instanciado para auxiliar na geração de texto a partir do modelo, usando o tokenizador especificado.








In [ ]:
inputs = tokenizer(
[
format_prompt.format(
        "Tell me more about",
        "The Long Ships",
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

In [ ]:
FastLanguageModel.for_inference(model)
generate = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

w<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Tell me more about

### Input:
The Long Ships

### Response:
The Long Ships is a novel by Frans G. Bengtsson, first published in Swedish in 1939. It is a humorous and epic tale of the Viking Age, focusing on the exploits of the Viking king Egil Skallagrimsson and his band of warriors. The book was translated into English by Alan Blair and published in 1958. It has been widely praised for its wit, humor, and vivid depiction of Viking life and culture. The Long Ships has been adapted into a film of the same name, as well as a television series.


In [ ]:
model.save_pretrained("/content/drive/MyDrive/FIAP/TC4/tc3_model_2409")
tokenizer.save_pretrained("/content/drive/MyDrive/FIAP/TC4/tc3_model_2409")

('/content/drive/MyDrive/FIAP/TC4/tc3_model_2409/tokenizer_config.json',
 '/content/drive/MyDrive/FIAP/TC4/tc3_model_2409/special_tokens_map.json',
 '/content/drive/MyDrive/FIAP/TC4/tc3_model_2409/tokenizer.model',
 '/content/drive/MyDrive/FIAP/TC4/tc3_model_2409/added_tokens.json',
 '/content/drive/MyDrive/FIAP/TC4/tc3_model_2409/tokenizer.json')

## Importando nosso modelo finetunado

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/FIAP/TC4/tc3_model_2409", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)


==((====))==  Unsloth 2024.9.post3: Fast Mistral patching. Transformers = 4.45.1.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

Unsloth: Will load /content/drive/MyDrive/FIAP/TC4/tc3_model_2409 as a legacy tokenizer.
Unsloth 2024.9.post3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
inputs = tokenizer(
[
format_prompt.format(
        "Tell me more about",
        "Giants Bread",
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

In [ ]:
generate = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<s>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Tell me more about

### Input:
Giants Bread

### Response:

Giants Bread is a 1933 novel by John Steinbeck. It is the story of a young man named Lennie Small, who has an intellectual disability, and his friend George Milton, who takes care of him. The two men travel around California during the Great Depression, looking for work and trying to survive. Lennie is a gentle giant, but he has a violent temper when he is frightened or angry. He also has a habit of crushing things in his hands, which leads to trouble for him and George. The novel is a classic of American literature, and it has been adapted
